In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("../input/train.csv")
data.head()

In [ ]:
data.describe()

Don't have any binary value. This dataset is too complicated

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
data.corr()

İt's too complicated also all values are pretty small.

In [ ]:
cormatrix=data.corr()

In [ ]:
cormatrix.max()

offfff

In [ ]:
target=data["TARGET"]
for col in data.columns:
    column=data[col]
    temp=pd.concat([target,column], axis=1)
    print(temp.corr())

Not to much correlation with target my friend.

In [ ]:
type(data["TARGET"])

## Dimension Reduction we have to do this

In [ ]:
data.shape

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50)
x=data.iloc[:,1:370]
y=data.iloc[:,370]
X=pca.fit_transform(x)
xfrm=pd.DataFrame(X)
type(xfrm)

In [ ]:
y.describe()

0 or 1 but describe do some interesting thing. First we count target

In [ ]:
xfrm.head()

Used XGBoost for classification

In [ ]:
from xgboost import XGBClassifier
cf=XGBClassifier()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xfrm, y, test_size=0.25, random_state=4200)
from sklearn.model_selection import cross_val_score
accuries = cross_val_score(estimator=cf,X=X_train, y = y_train,cv=10)
print(accuries)

XGBoost for well in training data. What about test data?

In [ ]:
test=pd.read_csv("../input/test.csv")
test.head()

In [ ]:
test.shape

In [ ]:
tx=data.iloc[:,1:370]
TX=pca.transform(tx)
TXFRM=pd.DataFrame(TX)
TXFRM.head()

In [ ]:
cf.fit(X_train,y_train)
ypred=cf.predict(TXFRM)
print(ypred)

In [ ]:
len(ypred)

In [ ]:
zerocount=0
for k in ypred:
    if k==0:
        zerocount=zerocount+1
print(zerocount)
print(len(ypred)-zerocount)

Nearly all the time says 0 it's big problem my friend.

In [ ]:
ypred=cf.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(confusion_matrix(y_pred=ypred,y_true=y_test))
print(accuracy_score(y_pred=ypred,y_true=y_test))
print(classification_report(y_pred=ypred,y_true=y_test))

XGBOOSt can't work well. 

## İt's not a classification problem it's anomaly detection problem. I explain further with anomaly detection perspective

In [ ]:
from imblearn.over_sampling import SMOTE

# Resample the minority class. You can change the strategy to 'auto' if you are not sure.
sm = SMOTE(sampling_strategy='minority', random_state=15)

# Fit the model to generate the data.
oversampled_trainX, oversampled_trainY = sm.fit_sample(xfrm,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(oversampled_trainX, oversampled_trainY, test_size=0.25, random_state=4200)
cf=XGBClassifier()
cf.fit(X_train,y_train)
ypred=cf.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(confusion_matrix(y_pred=ypred,y_true=y_test))
print(accuracy_score(y_pred=ypred,y_true=y_test))
print(classification_report(y_pred=ypred,y_true=y_test))

Accuracy is down so many. But prediction is more blanced. Also it's better than %50 so it's good I thing.

In [ ]:
ypred=cf.predict(TXFRM.values)
print(ypred)

In [ ]:
zerocount=0
for k in ypred:
    if k==0:
        zerocount=zerocount+1
print(zerocount)
print(len(ypred)-zerocount)

## Test data is balanced but this test also isn't balanced data.

Anomaly detection is interesting and complicated topic.